# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
# import libraries
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import TruncatedSVD
import nltk
import pickle
nltk.download(['punkt', 'wordnet'])

In [ ]:
# load data from database and transform data accordingly 
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('Dataset', engine)
X = df['message']
Y = df.iloc[:,4:]
Y.columns

### 2. Write a tokenization function to process your text data

In [ ]:
#create tokenization function
def tokenize(text):
    '''
    Function to clean, tokenize & lemmatize sentences
    Input: text (sentence9)
    Output: tokenized sentence
    '''
    text= re.sub(r"[^a-zA-Z0-9]", " ",text.lower()) #normalize data
    tokens = word_tokenize(text) # tokenize data
    lemmatizer = WordNetLemmatizer() # initialize lemmatizer
    clean_tokens = []
    for tok in tokens: # lemmatize data
        clean_tok = lemmatizer.lemmatize(tok).strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

In [ ]:
#check if function works
sentence = tokenize(X[1])
sentence

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
#create ML_Pipeline
model_pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
#Split data in training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size = 0.2, random_state = 45)
# train classifier
model_pipeline.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
def test_performance(model, X_test, y_test):
    '''
    Function to evaluate the model_pipeline
    Input: Model & test set
    Output: Classification performance of model
    '''
    y_pred = model_pipeline.predict(X_test)
    for i, col in enumerate(y_test):
        print(col)
        print(classification_report(y_test[col], y_pred[:, i]))

In [ ]:
#check if function works
test_performance(model_pipeline, X_test, y_test)

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
# create grid
parameters =  {'tfidf__use_idf': (True, False), 
              'clf__estimator__n_estimators': [30, 90], 
              'clf__estimator__min_samples_split': [2, 5]} 
# perform gridsearch
cv = GridSearchCV(model_pipeline, param_grid=parameters, n_jobs=-1)

In [ ]:
#check result
cv

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
#Please note, that cv.fit is resulting in an endless loop, I could not resolve this, hence gridsearch hyperparameter optimization will not be included
## run model_pipeline against testset
cv.fit(X_train, y_train)
test_performance(cv, X_test, y_test)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
#Improve model_pipeline
model_pipeline_2 = Pipeline([
    ('vect', CountVectorizer()),
    ('best', TruncatedSVD()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

In [ ]:
#Please note, that cv.fit is resulting in an endless loop, I could not resolve this, hence gridsearch hyperparameter optimization will not be included
#Train & predict
model_pipeline_2.fit(X_train, y_train)
test_performance(cv, X_test, y_test)(model_pipeline_2, X_test, y_test)

### 9. Export your model as a pickle file

In [ ]:
with open('model.pkl', 'wb') as f:
    pickle.dump(cv2, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.